In [1]:
from utils import load_data
import pandas as pd
from autogluon.tabular import TabularPredictor
import pandas as pd
import json
import h5py
import pickle
import numpy as np
from utils import get_train_test_datasets

/home/julian/miniconda3/envs/dtiam-esm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
included_model_types = ['NN_TORCH', 'FASTAI', 'GBM', 'XGB']
hyperparameters = {
	'NN_TORCH': [{'ag_args_fit': {'num_gpus': 1}}],
	'GBM': [
        {'ag_args_fit': {'num_gpus': 1}},
        {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'ag_args_fit': {'num_gpus': 1}, 'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'XGB': [{'ag_args_fit': {'num_gpus': 1}}],
	'FASTAI': [{'ag_args_fit': {'num_gpus': 1}}]
}
eval_metric = 'rmse'
preset=None

In [ ]:
dataset = 'kiba'
setting = 'warm_start'
target = 'dta'
# setting = 'drug_coldstart'
# setting = 'protein_coldstart'
for fold in range(1, 5):
    dataset_path = "../data/dta/" + dataset
    folds_path = dataset_path + f"/data_folds/{setting}/"

    mol_model = 'MolE_GuacaMol_27113.ckpt'
    prot_model = 'esmc_600m'
    # mol_model = 'dtiam_kiba_compound_features'
    train_df, test_df = get_train_test_datasets(fold=fold, setting=setting, dataset=dataset, target=target, mol_model=mol_model, prot_model=prot_model)
    model_path = f"/home/julian/DTIAM/code/AutogluonModels/{dataset}-{setting}-fold_{fold}-{mol_model.split('.')[0]}-{prot_model}"
    
    predictor = TabularPredictor(path=model_path, label="y", eval_metric=eval_metric, problem_type='regression').fit(
        train_data=train_df, included_model_types=included_model_types, presets=preset, hyperparameters=hyperparameters, fit_strategy='parallel'
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          20
Memory Avail:       27.35 GB / 62.63 GB (43.7%)
Disk Space Avail:   161.44 GB / 1684.96 GB (9.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with f

In [ ]:
# ag-20250616_230514 es drug_coldstart

In [16]:
# predictor.path = mol_model

In [8]:
test_data_nolab = test_df.drop(columns=["y"])

In [9]:
pred_scores = predictor.predict(test_data_nolab)

In [10]:
import numpy as np
from utils import load_data, rmse, mse, pearson, spearman, ci, roc_auc, pr_auc
res_all = pd.DataFrame(columns=["RMSE", "MSE", "Pearson", "Spearman", "CI"])
G, P = np.array(test_df["y"]), np.array(pred_scores)
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

In [11]:
i = 0
print(
    f"fold: {i+1}, RMSE: {ret[0]}, MSE: {ret[1]}, Pearson: {ret[2]}, Spearman: {ret[3]}, CI: {ret[4]}"
)
res_all.loc[i] = ret

fold: 1, RMSE: 0.5849176988608807, MSE: 0.342128714440708, Pearson: 0.7199550158109927, Spearman: 0.638433401990211, CI: 0.7514334587237242
